In [1]:
import molsysmt as msm
from molsysmt import pyunitwizard as puw
import omembrane as omem
import openmm as mm
from openmm import app as app
from openmm import unit as u
import numpy as np
import math
from matplotlib import pyplot as plt
from tqdm import tqdm
from sys import stdout

from openmm import LocalEnergyMinimizer

In [2]:
molsys = msm.convert('memb_popc_100_stage_2_4.h5msm')

In [3]:
P_atoms = msm.select(molsys, selection='atom_type=="P" and molecule_type=="lipid"')
N_atoms = msm.select(molsys, selection='atom_type=="N" and molecule_type=="lipid"')
O_atoms = msm.select(molsys, selection='atom_type=="O" and molecule_type=="lipid"')
C_atoms = msm.select(molsys, selection='atom_type=="C" and molecule_type=="lipid"')
OW_atoms = msm.select(molsys, selection='atom_type=="O" and molecule_type=="water"')

PO_heads_atoms = msm.select(molsys, selection='atom_name in ["P","O11","O12","O13","O14"] and molecule_type=="lipid"')

In [4]:
topology = msm.convert(molsys, 'openmm.topology')
positions = msm.get(molsys, coordinates=True)

In [5]:
forcefield = mm.app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [6]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME, nonbondedCutoff=1.2*u.nanometer, constraints=app.HBonds)

In [7]:
friction = 1 / u.picosecond
timestep = 2 * u.femtoseconds
temperature = 0 * u.kelvin
integrator = mm.LangevinIntegrator(temperature, friction, timestep)

In [8]:
platform = mm.Platform.getPlatformByName("CUDA")

In [9]:
simulation = app.Simulation(topology, system, integrator, platform)
simulation.context.setPositions(msm.pyunitwizard.convert(positions[0], to_form='openmm.unit'))

In [10]:
msm.thirds.openmm.forces.harmonic_potential_to_coordinates(simulation, selection=PO_heads_atoms,
                                                          force_constant=1000.0*u.kilojoules_per_mole/(u.nanometer**2),
                                                          pbc=True, adding_force=True)

In [11]:
simulation.minimizeEnergy()

In [12]:
reporter = app.StateDataReporter(stdout, 10000, step=True, potentialEnergy=True, temperature=True)
simulation.reporters = [reporter]

In [13]:
simulation.integrator.setTemperature(50*u.kelvin)
simulation.step(50000)
simulation.integrator.setTemperature(100*u.kelvin)
simulation.step(50000)
simulation.integrator.setTemperature(150*u.kelvin)
simulation.step(50000)
simulation.integrator.setTemperature(200*u.kelvin)
simulation.step(50000)
simulation.integrator.setTemperature(250*u.kelvin)
simulation.step(50000)
simulation.integrator.setTemperature(300*u.kelvin)
simulation.step(250000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
10000,-839554.9521373888,50.144426720606624
20000,-842197.3447155138,50.46601876780765
30000,-843693.3290905138,50.251589027376404
40000,-844760.1064342638,50.164110233326035
50000,-845500.7411998888,50.075731927046796
60000,-816850.9540905138,99.62311213055166
70000,-820127.3505748888,99.7914062770469
80000,-822285.3974498888,100.5560200383334
90000,-823247.6083873888,99.62273017414032
100000,-824065.3290905138,99.76086434248563
110000,-792427.2314342638,149.94360885303522
120000,-795658.1708873888,150.3647116094637
130000,-796891.1025280138,149.8097881263508
140000,-798804.2314342638,150.8526689348283
150000,-799850.7958873888,150.13017914873436
160000,-758072.5576061388,200.37605657065907
170000,-757930.7509655138,199.76686398079914
180000,-758453.3720592638,200.73376873305816
190000,-761095.4326061388,199.59839927655474
200000,-761362.1357311388,199.63484700842162
210000,-707889.0419811388,249.61078629022606
220000,-709242.4130

KeyboardInterrupt: 

In [ ]:
state_init = context.getState(getPositions=True)

In [ ]:
msm.molecular_mechanics.get_potential_energy(context)

In [ ]:
LocalEnergyMinimizer.minimize(context)

In [ ]:
msm.molecular_mechanics.get_potential_energy(context)

In [ ]:
state_end = context.getState(getPositions=True)

In [ ]:
bins_edges_P_init, density_P_init = omem.analysis.get_lineal_density(molsys, selection = P_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_N_init, density_N_init = omem.analysis.get_lineal_density(molsys, selection = N_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_C_init, density_C_init = omem.analysis.get_lineal_density(molsys, selection = C_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_O_init, density_O_init = omem.analysis.get_lineal_density(molsys, selection = O_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_OW_init, density_OW_init = omem.analysis.get_lineal_density(molsys, selection=OW_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)


plt.rcParams["figure.figsize"] = (6,4)
plt.plot((bins_edges_P_init[1:]+bins_edges_P_init[:-1])/2, density_P_init, label='P')
plt.plot((bins_edges_N_init[1:]+bins_edges_N_init[:-1])/2, density_N_init, label='N')
plt.plot((bins_edges_C_init[1:]+bins_edges_C_init[:-1])/2, density_C_init, label='C')
plt.plot((bins_edges_O_init[1:]+bins_edges_O_init[:-1])/2, density_O_init, label='O')
plt.plot((bins_edges_OW_init[1:]+bins_edges_OW_init[:-1])/2, density_OW_init, label='OW')
plt.legend()

In [ ]:
bins_edges_P_end, density_P_end = omem.analysis.get_lineal_density(state_end, selection = P_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_N_end, density_N_end = omem.analysis.get_lineal_density(state_end, selection = N_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_C_end, density_C_end = omem.analysis.get_lineal_density(state_end, selection = C_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_O_end, density_O_end = omem.analysis.get_lineal_density(state_end, selection = O_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)
bins_edges_OW_end, density_OW_end = omem.analysis.get_lineal_density(state_end, selection=OW_atoms, axis = [0,0,1],
                                                       bins = 80, range="[-6.0, 6.0] nm", normalized=True)

plt.rcParams["figure.figsize"] = (6,4)
plt.plot((bins_edges_P_end[1:]+bins_edges_P_end[:-1])/2, density_P_end, label='P')
plt.plot((bins_edges_N_end[1:]+bins_edges_N_end[:-1])/2, density_N_end, label='N')
plt.plot((bins_edges_C_end[1:]+bins_edges_C_end[:-1])/2, density_C_end, label='C')
plt.plot((bins_edges_O_end[1:]+bins_edges_O_end[:-1])/2, density_O_end, label='O')
plt.plot((bins_edges_OW_end[1:]+bins_edges_OW_end[:-1])/2, density_OW_end, label='OW')
plt.legend()
plt.show()

In [ ]:
shifts_P = msm.structure.get_distances(molecular_system=state_init, molecular_system_2=state_end,
                                       selection=P_atoms, pairs=True, pbc=True)
shifts_N = msm.structure.get_distances(molecular_system=state_init, molecular_system_2=state_end,
                                       selection=N_atoms, pairs=True, pbc=True)
shifts_C = msm.structure.get_distances(molecular_system=state_init, molecular_system_2=state_end,
                                       selection=C_atoms, pairs=True, pbc=True)
shifts_O = msm.structure.get_distances(molecular_system=state_init, molecular_system_2=state_end,
                                       selection=O_atoms, pairs=True, pbc=True)
shifts_OW = msm.structure.get_distances(molecular_system=state_init, molecular_system_2=state_end,
                                       selection=OW_atoms, pairs=True, pbc=True)

h_P, bins = np.histogram(puw.get_value(shifts_P[0]), range=[0.0, 1.0], bins=20, density=True)
h_N, bins = np.histogram(puw.get_value(shifts_N[0]), range=[0.0, 1.0], bins=20, density=True)
h_C, bins = np.histogram(puw.get_value(shifts_C[0]), range=[0.0, 1.0], bins=20, density=True)
h_O, bins = np.histogram(puw.get_value(shifts_O[0]), range=[0.0, 1.0], bins=20, density=True)
h_OW, bins = np.histogram(puw.get_value(shifts_OW[0]), range=[0.0, 1.0], bins=20, density=True)

plt.plot(0.5*(bins[1:]+bins[:-1]), h_P, label='P')
plt.plot(0.5*(bins[1:]+bins[:-1]), h_N, label='N')
plt.plot(0.5*(bins[1:]+bins[:-1]), h_C, label='C')
plt.plot(0.5*(bins[1:]+bins[:-1]), h_O, label='O')
plt.plot(0.5*(bins[1:]+bins[:-1]), h_OW, label='OW')
plt.legend()
plt.show()

In [ ]:
forces = msm.molecular_mechanics.get_forces(context, norm=True)
h, bins = np.histogram(puw.get_value(forces), bins=100, density=True)
plt.plot(0.5*(bins[1:]+bins[:-1]), h)
plt.show()

In [ ]:
#memb_antes = msm.extract(molsys, selection='molecule_type=="lipid"', to_form='molsysmt.MolSys')
#memb_despues = msm.extract([molsys, context], selection='molecule_type=="lipid"', to_form='molsysmt.MolSys')

In [ ]:
#msm.view(memb_despues)

In [ ]:
coordinates = msm.get(state_end, coordinates=True)
msm.set(molsys, coordinates=coordinates)

In [ ]:
msm.convert(molsys, 'memb_popc_100_stage_2_4.h5msm')